##  重试解析器

虽然在某些情况下，可以仅通过查看输出来修复任何解析错误，但在其他情况下则不然。例如，输出不仅格式不正确，而且部分完整。考虑下面的例子。

In [32]:
import pprint
from langchain.output_parsers import (
    OutputFixingParser,
    PydanticOutputParser,
)
from langchain.prompts import (
    PromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)

In [33]:
template = """根据用户问题，提供应采取的操作和操作输入。
{format_instructions}
问题: {query}
答复:"""


class Action(BaseModel):
    action: str = Field(description="采取的行动")
    action_input: str = Field(description="输入到动作")


parser = PydanticOutputParser(pydantic_object=Action)

In [34]:
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"action": {"title": "Action", "description": "\\u91c7\\u53d6\\u7684\\u884c\\u52a8", "type": "string"}, "action_input": {"title": "Action Input", "description": "\\u8f93\\u5165\\u5230\\u52a8\\u4f5c", "type": "string"}}, "required": ["action", "action_input"]}\n```'

In [35]:
prompt = PromptTemplate(
    template="回答用户的询问。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [36]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [38]:
testLLM = prompt | chat 
testLLM.invoke({"query": "who is leo di caprios gf?"})

AIMessage(content='The given schema defines a JSON object with two required properties: `action` and `action_input`. The `action` property should be a string while the `action_input` property can also be a string.\n\nTo answer the question "who is Leo Di Caprio\'s gf?", we need to find information about his girlfriend. However, this information may change over time and cannot be accurately answered without an up-to-date source of information. Therefore, it would not fit into the provided schema as a valid JSON instance.\n\nIf you have specific information about Leo Di Caprio\'s current girlfriend that can be represented in the given schema format, please provide it, and I will try to generate a well-formatted JSON instance for it.')

In [37]:
chain = prompt | chat | parser
parser_output = chain.invoke({"query": "who is leo di caprios gf?"})

parser_output

OutputParserException: Invalid json output: The output should be a JSON instance that conforms to the given schema. However, since the question does not provide any specific information about Leo DiCaprio's girlfriend, it would not be possible to create an object that meets the required format.

In [39]:
bad_response = '{"action": "search"}'

In [40]:
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {'action': 'search'}. Got: 1 validation error for Action
action_input
  field required (type=value_error.missing)

In [41]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=chat)

In [42]:
fix_parser.parse(bad_response)

Action(action='search', action_input='example')

In [44]:
from langchain.output_parsers import RetryOutputParser
retry_parser = RetryOutputParser.from_llm(parser=parser, llm=chat)



In [45]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

OutputParserException: Invalid json output: The output schema requires a JSON object with specific properties and types. The provided completion does not meet these requirements. Please provide a valid JSON instance that conforms to the schema.